In [207]:
import xapian
import spacy
from collections import Counter, defaultdict
import numpy as np
import json
from allennlp.predictors.predictor import Predictor
DROPBOX_PATH = "/Users/neesergparajuli/Dropbox/"
dev_SET = DROPBOX_PATH+"Webtext/Data/test-unlabelled.json"
predictor = Predictor.from_path(DROPBOX_PATH+"Webtext/WSTA_fact_checker/allen3/model.tar.gz")
DATABASE = DROPBOX_PATH+"Webtext/Data/XxapianDatabase2"
db = xapian.Database(DATABASE)
nlp = spacy.load('en_core_web_sm')

# Preprocessing queries and sentences

In [178]:
def preprocess(fact):
    sent = nlp(fact)
    query = []
    sentence = []
    title = []
    for ent in sent.ents:
        query.append("text:" + '"' + ent.text + '"')
        query.append("title:" + '"' + ent.text + '"')
        title.append(ent.text)
    for chunk in sent.noun_chunks:
        title.append(chunk.text)
    for token in sent:
        if token.tag_ == "NN" or token.pos_ == "PROPN" or token.tag_ == "VBG":
            query.append("text:" +  token.text )
        sentence.append(token.text)
        
    if not query:
        query.append("text:" + sent[0].text)
    return (query,sentence, title, sent)

# Query the databases and process results

In [211]:
def get_unique_id(terms):
    for term in terms:
        if "_~s~_" in term:
            ID = term.split("_~s~_" )
            docId = ID[0][1:]
            return docId
    
    print("wtdf")
    return "wtf"

def my_own_queryparser(fact):
    new_fact = ''
    for f in fact:
        new_fact += f + ' '
    return new_fact

'''Given document retrieves the sentence in it. doc = [docId, document data]'''

def get_sentences(docs):
    doc_dict = defaultdict(dict)
    for doc in docs:
        sent = doc[1].split('*_*')[1:]
        for s in sent:
            if s[1] in "1234567890":
                doc_dict[doc[0]][int(s[0]+s[1])] = s[2:]
            else:
                doc_dict[doc[0]][int(s[0])] = s[2:]
    return doc_dict

def sort_doc(title, docs, processed, t_processed):
    title = " ".join(title.split('_'))
    s= 0
    for fact in t_processed:
        if fact in title:
            s+= (len(fact)/len(title))
    counts = Counter()
    i = 1
    for key in docs:
        i += 1
        for fact in (processed):
            if counts[key, docs[key]] == 0:
                counts[key, docs[key]] = 1
            if fact in docs[key]:
                counts[key, docs[key]] +=1
            else:
                counts[key, docs[key]] +=0
    
    return (s*sum(counts.values())/i , counts.most_common())

''' retrieves the best n best sentences from  m best docs'''
def get_best_sentences(querys ,docs, n_docs = 10, n_sentences = 20):
    best_docs = []
    vector = querys[3]
    for doc in docs:
        (doc_scores, sentence_score) = sort_doc(doc,docs[doc],querys[1], querys[2])
        scores = (doc_scores, doc, sentence_score)
        best_docs.append(scores)
    best_docs.sort(reverse =True)
    best_docs = best_docs[:n_docs]
    best_sent = Counter()
    for best in best_docs:
        for sent in best[2]:
            best_sent[(best[1], sent[0][0])] = sentence_similarity(vector, sent[0][1])*best[0]*sent[1]    
    best_normal = normalise(best_sent.most_common(n_sentences))
    return best_normal

###measures the sentence similarity using spacy's word2vec embedding
def sentence_similarity(sentence_vector, fact):
    sent = nlp(fact)
    return sentence_vector.similarity(sent)

###this function normalises the relevance score
def normalise(best_n):
    best = best_n[0][1]
    best_normal = []
    for n,value in best_n:
        best_normal.append((n,value/best))
    return best_normal

In [212]:

'''Querys = [query, sentence tokens, title tokens, fact vector]'''
def get_docs(querys, k_matches):
    fact = querys[0]
    queryparser = xapian.QueryParser()
    queryparser.set_stemming_strategy(xapian.QueryParser.STEM_NONE)
    queryparser.add_prefix("text", "")
    queryparser.add_prefix("title", "S")
    query = queryparser.parse_query(my_own_queryparser(fact))
    enquire = xapian.Enquire(db)
    enquire.set_query(query)
    results = []
    for match in enquire.get_mset(0,k_matches):
        terms = match.document.termlist()
        unid = get_unique_id([term.term.decode("utf8") for term in terms])
        results.append((unid, match.document.get_data().decode("utf8")))
    if not results:
        return None
    docs = get_sentences(results)
    best_matches = get_best_sentences(querys, docs)
    final_result = []
    for match in best_matches:
        doc_id, sent_id = match[0]
        final_result.append(((doc_id, sent_id), docs[doc_id][sent_id], match[1]))
    return final_result

In [213]:
with open(dev_SET) as file:
    data = json.load(file)
# data ={}
# with open("testoutput", "w") as file:
#     for ob in data_:
#         if np.random.uniform()<1.0:
#             data[ob] = data_[ob]
#     json.dump(data, file, indent =4)

In [214]:
def get_topN(fact, n_fact):
    processed = preprocess(fact)
    premises = get_docs(processed, n_fact)
    facts_prob ={"SUPPORTS":[], "REFUTES":[]}
    if not premises:
        return facts_prob
    i=0
    for premise in premises:
        probs = predictor.predict(premise = premise[1], hypothesis = fact)["label_probs"]
        (prob1, prob2) = (probs[1], probs[2])
        if i >5:
            break
        if(prob1>0.5 and premise[2] > 0.6):
            facts_prob["SUPPORTS"].append((prob1,premise[0]))
        if(prob2>0.5 and premise[2] > 0.6):
            facts_prob["REFUTES"].append((prob2, premise[0]))
        i +=1
    return facts_prob

In [215]:
m = 0
empty = 0
js_dict = {}
print(len(data))
import time
start = time.time()
for ob in data:
    m+=1
    print(m)
    fact = data[ob]["claim"]
    premises = get_topN(fact, 100)
    refutes = premises["REFUTES"]
    supports = premises["SUPPORTS"]
    if len(refutes) == len(supports):
        js_dict[ob] = {"claim": fact, "label": "NOT ENOUGH INFO", "evidence": []}
    elif len(refutes) > len(supports):
        js_dict[ob] = {"claim": fact, "label": "REFUTES", "evidence": [ [ref[1][0], ref[1][1]] for ref in refutes ]}
    else:
        js_dict[ob] = {"claim": fact, "label": "SUPPORTS", "evidence": [ [sup[1][0], sup[1][1]] for sup in supports ]}
with open("testoutput.json","w") as file:
    json.dump(js_dict, file, indent=4)
print(time.time() - start)

14997
1
2
3
4
5
6
7
8
9


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
28

1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703


...

8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620


10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
1021

11416
11417
11418
11419
11420
11421
11422
11423
11424
11425
11426
11427
11428
11429
11430
11431
11432
11433
11434
11435
11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
11448
11449
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
11580
11581
1158

12782
12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
1294

14148
14149
14150
14151
14152
14153
14154
14155
14156
14157
14158
14159
14160
14161
14162
14163
14164
14165
14166
14167
14168
14169
14170
14171
14172
14173
14174
14175
14176
14177
14178
14179
14180
14181
14182
14183
14184
14185
14186
14187
14188
14189
14190
14191
14192
14193
14194
14195
14196
14197
14198
14199
14200
14201
14202
14203
14204
14205
14206
14207
14208
14209
14210
14211
14212
14213
14214
14215
14216
14217
14218
14219
14220
14221
14222
14223
14224
14225
14226
14227
14228
14229
14230
14231
14232
14233
14234
14235
14236
14237
14238
14239
14240
14241
14242
14243
14244
14245
14246
14247
14248
14249
14250
14251
14252
14253
14254
14255
14256
14257
14258
14259
14260
14261
14262
14263
14264
14265
14266
14267
14268
14269
14270
14271
14272
14273
14274
14275
14276
14277
14278
14279
14280
14281
14282
14283
14284
14285
14286
14287
14288
14289
14290
14291
14292
14293
14294
14295
14296
14297
14298
14299
14300
14301
14302
14303
14304
14305
14306
14307
14308
14309
14310
14311
14312
14313
1431

In [216]:
v = get_topN("Pharrell Williams is not a musician.", 100)
print(v)
for d in v:
    print(v[d])

{'SUPPORTS': [], 'REFUTES': [(0.9631324410438538, ('Pharrell_Williams', 11)), (0.946406900882721, ('Pharrell_Williams', 8)), (0.9927253723144531, ('Pharrell_Williams', 15)), (0.968196451663971, ('Pharrell_Williams', 0)), (0.9419309496879578, ('Pharrell_Williams', 14)), (0.9767147898674011, ('Pharrell_Williams', 4))]}
[]
[(0.9631324410438538, ('Pharrell_Williams', 11)), (0.946406900882721, ('Pharrell_Williams', 8)), (0.9927253723144531, ('Pharrell_Williams', 15)), (0.968196451663971, ('Pharrell_Williams', 0)), (0.9419309496879578, ('Pharrell_Williams', 14)), (0.9767147898674011, ('Pharrell_Williams', 4))]
